In [1]:
from random import randint

In [2]:
import tensorflow as tf
import os
import re
import datetime

/home/thomas/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/thomas/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
input_folder = 'data/processed'

In [4]:
model_folder = 'output'

In [5]:
from text_cnn import TextCNN

In [6]:
sess = tf.InteractiveSession()

In [7]:
SAMPLE_LENGTH = 200

In [8]:
BATCH_SIZE = 10

In [9]:
dropout_keep_prob = 0.5

In [10]:
evaluate_every = 100
checkpoint_every = 100
num_checkpoints = 5


In [11]:
out_dir = 'runs'
num_checkpoints = 10

In [12]:
import gzip, pickle as pkl
with gzip.GzipFile("data/abridged_index2word.pkl.gz", "rb") as f:
    abridged_index2word = pkl.load(f)
with gzip.GzipFile("data/abridged_word2index.pkl.gz", "rb") as f:
    abridged_word2index = pkl.load(f)

In [13]:
X_train = []
y_train_text = []
for root, folder, files in os.walk(input_folder):
    for file_name in files:
        if file_name == "README.md":
            continue
        with gzip.open(input_folder + '/' + file_name, 'rb') as f:
            all_token_ids = pkl.load(f)
        cat = re.sub("_.+", "", file_name)
        X_train.append(all_token_ids)
        y_train_text.append(cat)

In [14]:
id_to_cat = list(sorted(set(y_train_text)))
cat_to_id = {}
for idx, cat in enumerate(id_to_cat):
    cat_to_id[cat] = idx
cat_to_id

{'Anne': 0, 'Charlotte': 1, 'Emily': 2}

In [15]:
id_to_cat

['Anne', 'Charlotte', 'Emily']

In [16]:
with gzip.GzipFile("data/id_to_cat.pkl.gz", "wb") as f:
    pkl.dump(id_to_cat, f)

In [17]:
y_train = [cat_to_id[cat] for cat in y_train_text]
y_train

[1, 1, 0, 0, 1, 2, 1]

In [18]:
tf.reset_default_graph()

In [19]:
cnn = TextCNN(
            sequence_length=SAMPLE_LENGTH,
            num_classes=len(id_to_cat),
            vocab_size=len(abridged_index2word),
            embedding_size=300,
            filter_sizes=[3,4,5],
            num_filters=128)

In [20]:
sess = tf.InteractiveSession()

In [21]:
gensim_weights = [x for x in tf.global_variables() if "embedding" in str(x)]
other_weights = [x for x in tf.global_variables() if "embedding" not in str(x)]

In [22]:
saver_embedding = tf.train.Saver(gensim_weights)

In [23]:
global_step = tf.Variable(0, name="global_step", trainable=False)
optimizer = tf.train.AdamOptimizer(1e-3)
grads_and_vars = optimizer.compute_gradients(cnn.loss)
train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

In [24]:
# Keep track of gradient values and sparsity (optional)
grad_summaries = []
for g, v in grads_and_vars:
    if g is not None:
        grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
        sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        grad_summaries.append(grad_hist_summary)
        grad_summaries.append(sparsity_summary)
grad_summaries_merged = tf.summary.merge(grad_summaries)

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

In [25]:
# Summaries for loss and accuracy
loss_summary = tf.summary.scalar("loss", cnn.loss)
#acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

In [26]:
# Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
checkpoint_prefix = os.path.join(checkpoint_dir, "model")
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
saver = tf.train.Saver(other_weights, max_to_keep=num_checkpoints, keep_checkpoint_every_n_hours=1)

In [27]:
# Initialize all variables
sess.run(tf.global_variables_initializer())

In [28]:
saver_embedding.restore(sess, model_folder + "/gensim_weights")

INFO:tensorflow:Restoring parameters from output/gensim_weights


In [29]:
# Train Summaries
train_summary_op = tf.summary.merge([loss_summary,
                                     #acc_summary,
                                     grad_summaries_merged])
train_summary_dir = os.path.join(out_dir, "summaries", "train")
train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

In [30]:
def train_step(x_batch, y_batch, f):
    """
    A single training step
    """
    feed_dict = {
      cnn.input_x: x_batch,
      cnn.input_y: y_batch,
      cnn.dropout_keep_prob: dropout_keep_prob
    }
    '''
    _, step, summaries, loss, accuracy = sess.run(
        [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
        feed_dict)
    '''
    _, step, summaries, loss = sess.run(
        [train_op, global_step, train_summary_op, cnn.loss],
        feed_dict)
    time_str = datetime.datetime.now().isoformat()
    print("{}: step {}, loss {:g}, acc".format(time_str, step, loss
                                                    #, accuracy
                                                   ))
    train_summary_writer.add_summary(summaries, step)
    f.write(time_str + "\t" + str(step) + "\t" + str(loss) + "\n")
    f.flush()

In [31]:
from random import shuffle

In [32]:
X_train

[array([ 360,    2,    3, ...,   15, 1016,    0]),
 array([ 360,    2,    0, ...,   82,  278, 8120]),
 array([ 360,    2,    1, ...,   25,   38, 1363]),
 array([ 360,    2,    3, ..., 3118,    0,    0]),
 array([ 360,    2,    3, ...,    4,    1, 6687]),
 array([  3, 360,   2, ..., 284, 770,   3]),
 array([360,   2,   3, ...,   0,   0,   0])]

In [33]:
with open(model_folder + "/loss.txt", "a") as f:
    for epoch in range(1000000):
        segments = list(range(len(X_train)))
        shuffle(segments)
        
        for segment in segments:
            X_batch = X_train[segment]
            start_poss = []
            for i in range(BATCH_SIZE):
                start_pos = randint(0, len(X_batch) - SAMPLE_LENGTH - 1)
                start_poss.append(start_pos)
            
            X_data = [X_batch[start_pos:start_pos + SAMPLE_LENGTH] for start_pos in start_poss]
            y_data_single_segment = [int(a == y_train[segment]) for a in range(len(id_to_cat))]
            y_data = [y_data_single_segment for i in range(BATCH_SIZE)]
            train_step(X_data, y_data, f)
            current_step = tf.train.global_step(sess, global_step)

            if current_step % checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

2018-05-28T12:20:27.814028: step 1, loss 3.16074, acc
2018-05-28T12:20:27.962701: step 2, loss 1.96791, acc
2018-05-28T12:20:28.107753: step 3, loss 0.961665, acc
2018-05-28T12:20:28.249525: step 4, loss 7.27132, acc
2018-05-28T12:20:28.398177: step 5, loss 0.108345, acc
2018-05-28T12:20:28.553420: step 6, loss 5.58244, acc
2018-05-28T12:20:28.722935: step 7, loss 3.80542, acc
2018-05-28T12:20:28.895389: step 8, loss 3.79988, acc
2018-05-28T12:20:29.040770: step 9, loss 4.02268, acc
2018-05-28T12:20:29.182984: step 10, loss 3.81251, acc
2018-05-28T12:20:29.326429: step 11, loss 0.662819, acc
2018-05-28T12:20:29.464622: step 12, loss 0.95384, acc
2018-05-28T12:20:29.612314: step 13, loss 1.16995, acc
2018-05-28T12:20:29.758662: step 14, loss 1.63246, acc
2018-05-28T12:20:29.904422: step 15, loss 0.892508, acc
2018-05-28T12:20:30.050227: step 16, loss 2.18465, acc
2018-05-28T12:20:30.199986: step 17, loss 2.67578, acc
2018-05-28T12:20:30.349840: step 18, loss 2.55496, acc
2018-05-28T12:2

KeyboardInterrupt: 